In [1]:
# Dependencies
import pandas as pd
import numpy as py

In [2]:
# Load Files
school_data_path = "Resources/schools_complete.csv"
student_data_path = "Resources/students_complete.csv"

In [3]:
# Read Files
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

In [4]:
# Combine the data into a single dataset.
combined_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
combined_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
# District Summary

# Total Schools
tot_schools = len(combined_data["school_name"].unique())

# Total Students
tot_student = len(combined_data["Student ID"].unique())

# Total Budget
tot_budget = school_data["budget"].sum()

# Average Math Score
av_math_score = combined_data["math_score"].mean()

# Average Reading Score
av_reading_score = combined_data["reading_score"].mean()

# % Passing Math
sum_pass_math = (combined_data["math_score"]>70).sum()
percent_pass_math = (sum_pass_math/tot_student)*100

# % Passing Reading
sum_pass_reading = (combined_data["reading_score"]>70).sum()
percent_pass_reading = (sum_pass_reading/tot_student)*100

# Overall Passing Rate
percent_pass_overall = (percent_pass_math + percent_pass_reading)/2

In [6]:
dist_summary = pd.DataFrame({"Total Schools":[tot_schools], 
                                "Total Students":[tot_student], 
                                "Total Budget":[tot_budget], 
                                "Average Math Score":[av_math_score], 
                                "Average Reading Score":[av_reading_score],
                                "% Passing Math":[percent_pass_math],
                                "% Passing Reading":[percent_pass_reading], 
                                "% Overall Passing":[percent_pass_overall]})
dist_summary["Total Students"] = dist_summary["Total Students"].map("{:,}".format)
dist_summary["Total Budget"] = dist_summary["Total Budget"].map("${:,.2f}".format)
dist_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,72.392137,82.971662,77.681899


In [7]:
# School Name
by_school = combined_data.set_index('school_name').groupby(['school_name'])

# School Types
sch_types = school_data.set_index('school_name')['type']

# Total Number of Students
stu_per_sch = by_school['Student ID'].count()

# Total School Budget
sch_budget = school_data.set_index('school_name')['budget']

# Per Student Budget
stu_budget = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

# Average Math Score
avg_math = by_school['math_score'].mean()

# Average Reading Score
avg_read = by_school['reading_score'].mean()

# % Passing Math
pass_math = combined_data[combined_data['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch*100 

# % Passing Reading 
pass_read = combined_data[combined_data['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch*100 

# # Percentage Overall Passing
overall = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_sch*100 

sch_summary = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Per Student Budget": stu_budget,
    "Total School Budget": sch_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})



sch_summary = sch_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

# Format Columns
sch_summary.style.format({"Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$3,124,928",$628,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087",$581,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


In [8]:
# Top Performing Schools (By % Overall Passing)
top_schools = sch_summary.sort_values("Overall Passing Rate", ascending=False)

# Format Columns
top_schools.head().style.format({"Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130",$638,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574",$578,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


In [9]:
# Bottom Performing Schools (By % Overall Passing)
bot_schools = sch_summary.sort_values(['Overall Passing Rate'], ascending=True)
bot_schools.head()

bot_schools.head().style.format({'Total School Budget': "${:,}", 
                          'Per Student Budget': "${:.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363",$637,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,53.539172


In [10]:
# Average Math Score for students of each grade level

av_math_9th_grade = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
av_math_10th_grade = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
av_math_11th_grade = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
av_math_12th_grade = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

# Create a dataframe for math at various grade levels

average_math = pd.merge(av_math_9th_grade, av_math_10th_grade, on=["school_name"])
average_math = average_math.rename(columns = {"math_score_x":"9th", "math_score_y":"10th"})
average_math = pd.merge(average_math, av_math_11th_grade, on=["school_name"])
average_math = average_math.rename(columns = {"math_score":"11th"})
average_math = pd.merge(average_math, av_math_12th_grade, on=["school_name"])
average_math = average_math.rename(columns = {"math_score":"12th"})
 
                             
average_math

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [11]:
# Average Reading Score for students of each grade level

av_reading_9th_grade = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
av_reading_10th_grade = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
av_reading_11th_grade = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
av_reading_12th_grade = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

# Create a dataframe for reading at various grade levels
average_reading = pd.merge(av_reading_9th_grade, av_reading_10th_grade, on=["school_name"])
average_reading = average_reading.rename(columns = {"reading_score_x":"9th", "reading_score_y":"10th"})
average_reading = pd.merge(average_reading, av_reading_11th_grade, on=["school_name"])
average_reading = average_reading.rename(columns = {"reading_score":"11th"})
average_reading = pd.merge(average_reading, av_reading_12th_grade, on=["school_name"])
average_reading = average_reading.rename(columns = {"reading_score":"12th"})

average_reading

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [20]:
# Scores by School Spending

#Create Bins for Table

bins = [0, 584.999, 614.999, 644.999, 999999]
group_name = ['< $585', "$585 - 614", "$615 - 644", "$645 - 680"]
combined_data['spending_bins'] = pd.cut(combined_data['budget']/combined_data['size'], bins, labels = group_name)

#group by spending
by_spending = combined_data.groupby('spending_bins')

average_math = by_spending['math_score'].mean()
average_reading = by_spending['reading_score'].mean()
pass_math = combined_data[combined_data['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()*100
pass_read = combined_data[combined_data['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()*100
overall_passing = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()*100

# Build Table
scores_by_spend = pd.merge(average_math, average_reading, on=["spending_bins"])
scores_by_spend = scores_by_spend.rename(columns = {"math_score":"Average Math Score", "reading_score":"Average Reading Score"})

scores_by_spend = pd.merge(scores_by_spend, pass_math, on=["spending_bins"])
scores_by_spend = scores_by_spend.rename(columns = {"Student ID":"% Passing Math"})

scores_by_spend = pd.merge(scores_by_spend, pass_read, on=["spending_bins"])
scores_by_spend = scores_by_spend.rename(columns = {"Student ID":"% Passing Reading"})

scores_by_spend = pd.merge(scores_by_spend, overall_passing, on=["spending_bins"])
scores_by_spend = scores_by_spend.rename(columns = {"Student ID":"% Overall Passing"})

scores_by_spend

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
spending_bins,,,,,
< $585,83.363065,83.964039,93.702889,96.686558,90.640704
$585 - 614,83.529196,83.838414,94.124128,95.886889,90.121190
$615 - 644,78.061635,81.434088,71.400428,83.614770,60.289317
$645 - 680,77.049297,81.005604,66.230813,81.109397,53.528791


In [13]:
# Scores by School Size
bins = [0, 999, 1999, 99999999999]
group_name = ["Small (<1000)", "Medium (1000-2000)", "Large (>2000)"]
combined_data['size_bins'] = pd.cut(combined_data['size'], bins, labels = group_name)

#group by school size
by_size = combined_data.groupby('size_bins')

average_math = by_size['math_score'].mean()
average_reading = by_size['reading_score'].mean()
pass_math = combined_data[combined_data['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()*100
pass_reading = combined_data[combined_data['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()*100
overall_passing = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()*100

# Build Table
scores_by_size = pd.merge(average_math, average_reading, on=["size_bins"])
scores_by_size = scores_by_size.rename(columns = {"math_score":"Average Math Score"})

scores_by_size = pd.merge(scores_by_size, pass_math, on=["size_bins"])
scores_by_size = scores_by_size.rename(columns = {"Student ID":"% Passing Math"})

scores_by_size = pd.merge(scores_by_size, pass_reading, on=["size_bins"])
scores_by_size = scores_by_size.rename(columns = {"Student ID":"% Passing Reading"})

scores_by_size = pd.merge(scores_by_size, overall_passing, on=["size_bins"])
scores_by_size = scores_by_size.rename(columns = {"Student ID":"% Overall Passing"})

scores_by_size


,Average Math Score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
size_bins,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (>2000),77.477597,81.198674,68.652380,82.125158,56.574046


In [14]:
# # Scores by School Type
#bins = [0, 999, 1999, 99999999999]
#group_name = ["Small (<1000)", "Medium (1000-2000)", "Large (>2000)"]
#combined_data['type_bins'] = pd.cut(combined_data['type'], bins, labels = group_name)

#group by school type
by_type = combined_data.groupby('type')

average_math = by_type['math_score'].mean()
average_reading = by_type['reading_score'].mean()
pass_math = combined_data[combined_data['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()*100
pass_reading = combined_data[combined_data['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()*100
overall_passing = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()*100

# Build Table
scores_by_type = pd.merge(average_math, average_reading, on=["type"])
scores_by_type = scores_by_type.rename(columns = {"math_score":"Average Math Score"})

scores_by_type = pd.merge(scores_by_type, pass_math, on=["type"])
scores_by_type = scores_by_type.rename(columns = {"Student ID":"% Passing Math"})

scores_by_type = pd.merge(scores_by_type, pass_reading, on=["type"])
scores_by_type = scores_by_type.rename(columns = {"Student ID":"% Passing Reading"})

scores_by_type = pd.merge(scores_by_type, overall_passing, on=["type"])
scores_by_type = scores_by_type.rename(columns = {"Student ID":"% Overall Passing"})

scores_by_type


,Average Math Score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878


TREND DESCRIPTION:

From the analysis above, we can observe the different variations in overall school performance based on different factors. First, we observe that overall size/population affects performance as small and medium-sized schools generally perform better than schools with large-sized schools with the former scoring an average of 90 percent and the latter scoring approximately 56 percent. 

Conversely, the overall passing falls with spending as schools that spend less than 615 dollars per student scored around 90 percent whilst schools that spent more than 615 dollars scored between 50 percent and 60 percent overall.

When schools are sorted by type, it is observed that Charter schools perform better than their district counterparts. Charter schools has around 90 percent  passing rate whilst District school scored around 50 percent.